# Import required packages


In [1]:
from pathlib import Path

import dtreeviz
import ee
import geemap
import geojson
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import plotly.express as px
import scipy.interpolate
import shap
from cuml.explainer import PermutationExplainer
from cuml.model_selection import train_test_split as cuml_train_test_split
from optuna_integration import LightGBMPruningCallback
from plotly import graph_objects as go
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    roc_auc_score,
)
from optuna.pruners import SuccessiveHalvingPruner
from optuna.samplers import GPSampler
from sklearn.model_selection import train_test_split

Second enviroment import due to environment conflict between RAPIDSAI and Pytorch-tabnet

In [ ]:
# import lightgbm as lgb
# import numpy as np
# import pandas as pd
# import torch
# from pytorch_tabnet.tab_model import TabNetClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import (
#     accuracy_score,
#     precision_recall_fscore_support,
#     roc_auc_score,
# )
# from sklearn.model_selection import train_test_split

In [4]:
ee.Authenticate()
ee.Initialize()

# Exploratory Data Analysis (EDA)


## Data Collection


Dataset used:

- [Global Flood Database v1 (2000-2018)](https://developers.google.com/earth-engine/datasets/catalog/GLOBAL_FLOOD_DB_MODIS_EVENTS_V1#description)

- [USGS Landsat 7 Level 2, Collection 2, Tier 1 ](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C02_T2_L2#description)

- [SRTM Digital Elevation Data Version 4 ](https://developers.google.com/earth-engine/datasets/catalog/CGIAR_SRTM90_V4)


In [ ]:
flood_collection: ee.ImageCollection = ee.ImageCollection(
    "GLOBAL_FLOOD_DB/MODIS_EVENTS/V1"
)

landsat_collection: ee.ImageCollection = ee.ImageCollection(
    "LANDSAT/LE07/C02/T1_L2"
).filterDate("2000-02-17", "2018-12-10")
elevation_image = ee.Image("CGIAR/SRTM90_V4")

print("Image structure of each collection:")
display(flood_collection.first(), landsat_collection.first(), elevation_image)

Mask the images with conditions:

- Clear percentage of flood images **>= 0.5**

- **Not** permanent water


In [ ]:
def mask_flood(image):
    clear_perc_mask = image.select("clear_perc").gte(0.5)  # Clear percentage >= 50%
    jrc_perm_water_mask = image.select("jrc_perm_water").eq(
        0
    )  # 0 - non-water | 1 - permanent water

    combined_mask = clear_perc_mask.And(jrc_perm_water_mask)

    masked_image = image.updateMask(combined_mask)

    return masked_image


flood_collection = flood_collection.map(mask_flood)

Compose the following bands into an image collection:

- **_x_**, **_y_**, _elevation_ as **_z_** and **_slope_**

- **_SR_B1, SR_B2, SR_B3, SR_B4, SR_B5, SR_B7_** from Landsat Collection

- **_NDWI_** calculated using SR_B2 and SR_B4

- **_duration, clear_perc, flooded_** from Global Flood Database


In [ ]:
landsat_interest_bands = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B7"]
flood_interest_bands = ["duration", "clear_perc", "flooded"]

elevation = elevation_image.select("elevation").rename("z")
slope = ee.Terrain.slope(elevation_image)


def filter_single_scene(image):
    date_range = ee.DateRange(
        image.get("system:time_start"), image.get("system:time_end")
    )
    return ee.Algorithms.If(
        date_range.end().difference(date_range.start(), "days").gt(0), image, None
    )


def compose_landsat(image):
    geometry = image.geometry()
    date_range = ee.DateRange(
        image.get("system:time_start"), image.get("system:time_end")
    )
    landsat_image = (
        landsat_collection.select(ee.List(landsat_interest_bands))
        .filterDate(date_range.start(), date_range.end())
        .filterBounds(geometry)
        .median()
    )
    return ee.Algorithms.If(
        landsat_image.bandNames().length(),
        image.select(flood_interest_bands).addBands(
            landsat_image,
            ee.List(landsat_interest_bands),
        ),
        None,
    )


def add_elevation_bands(image: ee.Image):
    return image.addBands(elevation).addBands(slope)


def set_default_projection(image: ee.Image):
    projection = image.select("flooded").projection()
    return image.setDefaultProjection(projection)


images = (
    flood_collection.map(filter_single_scene, True)
    .map(compose_landsat, True)
    .map(add_elevation_bands)
    .map(set_default_projection)
)
print(f"Number of flood event images left: {images.size().getInfo()}")
images.first()

In [ ]:
def to_features(image: ee.Image):
    # Reduce resolution of bands to a common scale
    image = image.addBands(
        image.select([*landsat_interest_bands, "z", "slope"]).reduceResolution(
            reducer=ee.Reducer.mean()
        ),
        overwrite=True,
    )
    return image.stratifiedSample(
        classBand="flooded",
        numPoints=50,
        dropNulls=True,
        scale=250,
        tileScale=4,
        region=image.geometry(),
        geometries=False,
        projection=image.projection(),
    )


dataset = images.map(to_features).flatten()

In [ ]:
# geemap.ee_export_vector_to_drive(dataset, description="flood_dataset", fileFormat="csv")

Extract the details of flood events


In [ ]:
interest_properties = {
    "system:index": "id",
    "dfo_country": "primary_country",
    "dfo_severity": "severity",
    "system:time_start": "start_date",
    "system:time_end": "end_date",
    "dfo_centroid_x": "center_lon",
    "dfo_centroid_y": "center_lat",
    "dfo_main_cause": "main_cause",
    "dfo_dead": "dead",
    "dfo_displaced": "displaced",
}


def extract_properties(image):
    old_keys = ee.List(list(interest_properties.keys()))
    new_keys = ee.List(list(interest_properties.values()))
    props: ee.Dictionary = image.toDictionary(old_keys).rename(old_keys, new_keys, True)
    mask = image.select("flooded").gt(0)
    area = (
        ee.Image.pixelArea()
        .mask(mask)
        .reduceRegion(
            reducer=ee.Reducer.sum(),
            scale=250,
            geometry=image.geometry(),
            maxPixels=412598311,
            crs=image.projection(),
        )
    ).get("area")
    props = props.set("area", area)
    return ee.Feature(None, props)


flood_properties = ee.FeatureCollection(flood_collection.map(extract_properties))

In [ ]:
# geemap.ee_to_csv(flood_properties, filename="../data/asg3/flood_props.csv")

## Data Cleaning


Checking for data distribution and null entry


In [ ]:
df_flood_props = pd.read_csv("../data/asg3/flood_props.csv", index_col="id").reset_index()
display(df_flood_props.head(), df_flood_props.describe(), df_flood_props.isna().sum())

In [ ]:
df_flood_data = pd.read_csv("../data/asg3/flood_dataset.csv", index_col="system:index").reset_index()
display(df_flood_data.head(), df_flood_data.describe(), df_flood_data.isna().sum())

In [ ]:
df_flood_data = pd.read_csv("../data/asg3/flood_dataset.csv", index_col="system:index").reset_index()
display(df_flood_data.head(), df_flood_data.describe(), df_flood_data.isna().sum())

In [ ]:
with open("countries.geojson") as f:
    countries_boundary = geojson.load(f)

countries_ADM = [
    feature["properties"]["ADMIN"] for feature in countries_boundary["features"]
]

Verify all country is valid with ADM standard


In [ ]:
def check_country_exists_ADM(countries):
    return pd.DataFrame(
        {
            "primary_country": countries,
            "exists_in_ADM": countries.isin(countries_ADM),
        }
    )


df_temp = check_country_exists_ADM(
    pd.Series(df_flood_props["primary_country"].unique())
)
df_temp[df_temp["exists_in_ADM"] == False]

Transform invalid countries' value according to ADM standard


In [ ]:
country_to_adm = {
    "USA": "United States of America",
    "UK": "United Kingdom",
    "Burma": "Myanmar",
    "Tanzania": "United Republic of Tanzania",
    "Columbia": "Colombia",
    "Bosnia-Herzegovina": "Bosnia and Herzegovina",
    "Guatamala": "Guatemala",
    "Serbia": "Republic of Serbia",
    "Africa": "Central African Republic",  # Africa is a continent, not a country
    "Texas": "United States of America",  # Texas is a state in the USA
    "Venezulea": "Venezuela",
    "Serbia-Montenegro": "Republic of Serbia",  # No longer exists as a single country
    "Inda": "India",
    "Democratic Republic of Congo": "Democratic Republic of the Congo",
    "Tasmania": "Australia",  # Tasmania is a state in Australia
    "The Gambia": "Gambia",
    "Scotland": "United Kingdom",  # Scotland is part of the United Kingdom
    "Gaza": "Palestine",  # Gaza is a region in Palestine
    "Kazahkstan": "Kazakhstan",
    "Uruguay,": "Uruguay",
    "Bahamas": "The Bahamas",
}


def process_country_name(country):
    processed_country = country_to_adm.get(country)
    return processed_country if processed_country else country


df_flood_props_cleaned = df_flood_props.copy()
df_flood_props_cleaned["primary_country"] = df_flood_props["primary_country"].apply(
    process_country_name
)
df_temp = check_country_exists_ADM(
    pd.Series(df_flood_props_cleaned["primary_country"].unique())
)
df_temp.sort_values("exists_in_ADM")

In [ ]:
# df_flood_props_cleaned.to_csv("../data/asg3/flood_props_cleaned.csv")

## Data Visualization


In [ ]:
causes = [x.lower() for x in df_flood_props_cleaned["main_cause"].to_list()]

categories = {
    "Heavy rain": ["rain", "monsoon", "torrential"],
    "Dam break/release": ["dam", "levy", "release"],
    "Snowmelt": ["snow"],
    "Ice jam": ["ice"],
    "Tropical storm": ["tropical", "typhoon", "hurricane"],
}

df_flood_cause = pd.DataFrame(
    data={
        "event": list(categories.keys()) + ["Other"],
        "count": np.zeros(len(categories) + 1, dtype=int),
        "matched": [[] for _ in range(len(categories) + 1)],
    }
)

for event in causes:
    matched = False
    for category, keywords in categories.items():
        if any(keyword in event for keyword in keywords):
            idx = df_flood_cause.index[df_flood_cause["event"] == category].tolist()[0]
            df_flood_cause.at[idx, "count"] += 1
            df_flood_cause.at[idx, "matched"].append(event)
            matched = True
    if not matched:
        idx = df_flood_cause.index[df_flood_cause["event"] == "Other"].tolist()[0]
        df_flood_cause.at[idx, "count"] += 1
        df_flood_cause.at[idx, "matched"].append(event)


fig = px.bar(
    df_flood_cause,
    x="event",
    y="count",
    color="count",
    text="count",
)
fig.update_layout(title=dict(text="Main Cause of Flood Events", x=0.5))
fig.show()
fig = px.pie(
    df_flood_data["flooded"].value_counts().to_frame().reset_index(),
    values="count",
    names=["Non-water", "Flooded"],
)
fig.update_layout(
    title=dict(text="Ratio of Flooded Area Against Non-Water Area in Dataset", x=0.5),
    margin_b=20,
)
fig.show()

Global flood occurrence (only primary influenced country)


In [ ]:
df_flood_by_country = pd.DataFrame(
    df_flood_props_cleaned["primary_country"].value_counts().reset_index(name="counts")
)

scattergeo_trace = go.Scattergeo(
    lat=df_flood_props_cleaned["center_lat"],
    lon=df_flood_props_cleaned["center_lon"],
    mode="markers",
    marker=dict(
        size=12,
        opacity=0.8,
        autocolorscale=False,
        symbol="triangle-down",
        colorscale="Reds",
        cmin=0,
        color=df_flood_props_cleaned["severity"],
        cmax=df_flood_props_cleaned["severity"].max(),
        colorbar_title="Severity",
        colorbar_x=1.15,
    ),
    hoverinfo="skip",
)

choropleth_trace = go.Choropleth(
    locations=df_flood_by_country["primary_country"],
    locationmode="country names",
    z=df_flood_by_country["counts"],
    colorscale="amp",
    colorbar_title="Occurence",
    hoverlabel_namelength=0,
)


fig = go.Figure(data=[scattergeo_trace, choropleth_trace])
fig.data[0].showlegend = False

fig.update_layout(
    title_text="Flood Severity and Occurrence",
    title_x=0.5,
    geo=dict(
        showland=True,
        landcolor="rgb(95, 138, 92)",
        showcountries=True,
        countrycolor="rgb(255, 255, 255)",
        showocean=True,
        oceancolor="rgb(158,202,225)",
        projection_type="orthographic",
    ),
    margin={"b": 15, "l": 20, "r": 0, "t": 70},
    shapes=list(
        [
            dict(
                fillcolor="rgb(95, 138, 92)",
                layer="below",
                line={"dash": "dash"},
                name="Country not primarily <br>affected by flood",
                showlegend=True,
                type="rect",
                xref="paper",
            )
        ]
    ),
    legend=dict(
        yanchor="top",
        y=1,
        xanchor="left",
        x=0,
        bgcolor="LightSteelBlue",
        bordercolor="Black",
        borderwidth=1,
    ),
    updatemenus=[
        {
            "buttons": [
                {
                    "args": [
                        {"geo.projection.type": "orthographic"},
                    ],
                    "label": "Orthographic",
                    "method": "relayout",
                },
                {
                    "args": [
                        {"geo.projection.type": "equirectangular"},
                    ],
                    "label": "Equirectangular",
                    "method": "relayout",
                },
            ],
            "direction": "left",
            "showactive": True,
            "type": "buttons",
            "x": 0,
            "xanchor": "left",
            "y": 1.15,
            "yanchor": "top",
        }
    ],
)
fig.show()

fig = px.scatter(
    df_flood_props_cleaned,
    x="displaced",
    y="dead",
    size="area",
    color="primary_country",
    hover_name="id",
)
fig.update_layout(
    title=dict(text="Estimated Displaced Against Fatalities Due to Flood Event", x=0.5),
    height=750,
)
fig.show()

Select the image of 2010 Pakistan floods for visualizing


In [ ]:
target_flood = "DFO_2507_From_20040620_to_20041007"
max_pixels = 10000
scale = 25000

target_image = (
    images.filter(ee.Filter.eq("system:index", target_flood)).first().unmask()
)

# Extract the image as features (x, y, z, flooded) to investigate the relationship between elevation/slope and flood
target_image_features = ee.FeatureCollection(
    target_image.reduceResolution(reducer=ee.Reducer.mean(), maxPixels=max_pixels)
    .addBands(ee.Image.pixelCoordinates(projection=target_image.projection()))
    .select(["x", "y", "z", "flooded"])
    .reduceRegion(
        ee.Reducer.toCollection(["x", "y", "z", "flooded"]),
        scale=scale,
        geometry=target_image.geometry().bounds(),
        crs=target_image.projection(),
    )
    .get("features")
)

display(target_image)
print(f"Number of points: {target_image_features.size().getInfo()}")

In [ ]:
# geemap.ee_to_csv(
#     target_image_features, filename=f"../data/asg3/{target_flood}_data.csv"
# )

Plot 3D DEM of target image with flood area colored


In [ ]:
target_image_data = pd.read_csv(f"../data/asg3/{target_flood}_data.csv").to_numpy()

flooded = target_image_data[:, 0]
x = target_image_data[:, 1]
y = target_image_data[:, 2]
z = target_image_data[:, -1]

xmin, xmax = np.min(x), np.max(x)
ymin, ymax = np.min(y), np.max(y)
xstep, ystep = (
    np.round((xmax - xmin) / np.unique(x).shape).item(),
    np.round((ymax - ymin) / np.unique(y).shape).item(),
)
u = np.arange(start=xmin, stop=xmax, step=xstep)
v = np.arange(start=ymin, stop=ymax, step=ystep)

X, Y = np.meshgrid(u, v)
Z = scipy.interpolate.griddata(
    (x, y), z, (X, Y), method="nearest", fill_value=0, rescale=True
)
F = scipy.interpolate.griddata(
    (x, y), flooded, (X, Y), method="nearest", fill_value=0, rescale=True
)
Fmin, Fmax = np.min(F), np.max(F)
fig = go.Figure(
    data=[
        go.Surface(
            z=Z, colorscale="Plotly3", showscale=True, colorbar_title="Elevation"
        ),
        go.Surface(
            z=Z + 150,
            surfacecolor=F,
            cmin=Fmin,
            cmax=Fmax,
            colorscale=[
                [0, "rgba(7, 148, 242, 0.0)"],
                [0.3, "rgba(7, 148, 242, 1.0)"],
                [1, "rgba(7, 148, 242, 1.0)"],
            ],
            colorbar=dict(title="Flood mean", x=-0.1),
        ),
    ]
)
fig.update_traces(
    contours_z=dict(
        show=True,
        usecolormap=True,
        project_z=True,
        color="rgba(255, 0, 0, 0.5)",
    )
)
fig.update_layout(
    scene_camera_eye=dict(x=1.5, y=1.5, z=1.25),
    margin=dict(l=0, r=0, t=0, b=30),
    title=dict(text=f"3D DEM of {target_flood}", x=0.5, y=0.95),
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(
                    label="Toggle Elevation Surface",
                    method="update",
                    args=[
                        {
                            "visible": [True, True],
                            "contours.z.usecolormap": [True, True],
                        },
                    ],
                    args2=[
                        {
                            "visible": [False, True],
                            "contours.z.usecolormap": [False, False],
                        },
                    ],
                ),
            ],
            direction="left",
            showactive=True,
            x=-0.1,
            xanchor="left",
            y=1.1,
            yanchor="top",
        )
    ],
)
fig.show()

Visualize dataset and target flood event with satellite view


In [ ]:
Map = geemap.Map()
Map.add_basemap("HYBRID")

vis_params = {"min": 0, "max": 10, "palette": ["c3effe", "1341e8", "051cb0", "001133"]}

Map.addLayer(
    flood_collection.select("jrc_perm_water").sum().gte(1).selfMask(),
    {"min": 0, "max": 1, "palette": "c3effe"},
    "JRC Permanent Water",
)

Map.addLayer(
    flood_collection.select("flooded").sum().selfMask(),
    vis_params,
    "GFD Satellite Observed Flood Plain",
)

Map.add_colorbar_branca(
    colors=vis_params["palette"],
    vmin=vis_params["min"],
    vmax=vis_params["max"],
    layer_name="",
    caption="Flood occurrence",
    discrete=True,
)

Map.addLayer(
    images.select("flooded").sum().selfMask(),
    vis_params,
    "Masked GFD Satellite Observed Flood Plain",
)

Map.addLayer(
    target_image.select("flooded").selfMask(),
    vis_params,
    "Selected flood event",
)
Map.addLayer(
    ee.FeatureCollection(ee.Feature(target_image.geometry().bounds())).style(
        color="red", fillColor="00000000"
    ),
    {},
    "Selected flood event boundary",
)

Map

# Feature Engineering


Initial selected features


In [ ]:
features = {
    "SR_B1": "Blue",
    "SR_B2": "Green",
    "SR_B3": "Red",
    "SR_B4": "NIR",
    "SR_B5": "SWIR_1",
    "SR_B7": "SWIR_2",
    "slope": "Slope",
    "z": "Elevation",
}
x = df_flood_data[features.keys()].rename(columns=features)
y = df_flood_data["flooded"]

## Feature Creation


Band Performance Comparison

- https://www.researchgate.net/publication/328853274_Remote_sensing_gis_and_hec-ras_techniques_applied_for_flood_extent_validation_based_on_landsat_imagery_lidar_and_hydrological_data_Case_study_Baseu_river_Romania
- https://www.sciencedirect.com/science/article/pii/S1110982321000703

MBI

- https://www.mdpi.com/2073-445X/10/3/231
- https://www.sciencedirect.com/science/article/pii/S2772375524000340

Transformed Index

- NDWI - Normalized Difference Water Index
- MBI - Modified Bare Soil Index
- NDVI - Normalized Difference Vegetation Index
- WRI - Water Ratio Index
- AWEI - Automated Water Extraction Index
- SI - Surface Index
- NWI - New Water Index


In [ ]:
x["NDWI"] = (x["Green"] - x["NIR"]) / (x["Green"] + x["NIR"])
x["MBI"] = (
    (x["SWIR_1"] - x["SWIR_2"] - x["NIR"]) / (x["SWIR_1"] + x["SWIR_2"] + x["NIR"])
) + 0.5
x["NDVI"] = (x["NIR"] - x["Red"]) / (x["NIR"] + x["Red"])
x["WRI"] = (x["Green"] + x["Red"]) / (x["NIR"] + x["SWIR_2"])
x["AWEI"] = (
    x["Blue"] + 2.5 * x["Green"] - 1.5 * (x["NIR"] + x["SWIR_1"]) - 0.25 * x["SWIR_2"]
)
x["SI"] = (1 - x["Red"]) * (1 - x["Blue"]) * (1 - x["Green"])
x["NWI"] = (x["Blue"] - (x["NIR"] + x["SWIR_1"] + x["SWIR_2"])) / (
    x["Blue"] + (x["NIR"] + x["SWIR_1"] + x["SWIR_2"])
)

Correlation heatmap of initial selected features


In [ ]:
corr = x.corr().round(4)
fig = px.imshow(corr, text_auto=True, aspect="auto")
fig.update_layout(title=dict(text="Correlation of Features", x=0.5))
fig.show()

Use LightGBM as base model for SHAP


In [ ]:
forest = lgb.LGBMClassifier(
    objective="binary",
    n_estimators=1000,
    learning_rate=0.01,
    deterministic=True,
    device_type="cpu",
    verbose=0,
)
forest

In [ ]:
X_train, X_test, y_train, y_test = cuml_train_test_split(
    x, y, test_size=0.3, shuffle=True, stratify=y
)

forest.fit(X_train.to_numpy(), y_train.to_numpy())

full_clustering_path = Path("../data/asg3/f_clustering.npy")
shap_values_path = Path("../data/asg3/shap_values.npy")

if full_clustering_path.exists():
    clustering = np.load(full_clustering_path)
else:
    clustering = shap.utils.hclust(X_train.to_numpy(), y_train.to_numpy())
    np.save(full_clustering_path, clustering)

if shap_values_path.exists():
    shap_values = np.load(shap_values_path)
else:
    masker = shap.maskers.Partition(X_train.to_numpy(), clustering=clustering)
    explainer = PermutationExplainer(
        forest.predict_proba, masker, data=X_train, masker_type="partition"
    )
    shap_values = explainer.shap_values(X_test.to_numpy())
    np.save(shap_values_path, shap_values)

In [ ]:
expected_value = np.mean(forest.predict_proba(X_test.to_numpy()), axis=0)

explanation = shap.Explanation(
    shap_values,
    data=X_test.to_numpy(),
    base_values=np.full(shape=np.sum(shap_values, axis=1).shape, fill_value=expected_value),
    feature_names=list(x.columns.values),
)

fig, axes = plt.subplots(1,2)

axes[0].set_title("Feature Importance and Clustering \nBased on SHAP Values")
shap.plots.bar(explanation[...,1], clustering=clustering, max_display=len(x.columns), ax=axes[0], show=False)

shap.decision_plot(expected_value[1], shap_values[...,1][:10], feature_names=list(x.columns.values), show=False)
axes[1] = plt.gca()
axes[1].set_title("Complexity of Model for Making Prediction")
fig.set_dpi(1000)
fig.suptitle("Analysis for All Features", fontsize=16)
fig.tight_layout()
fig.set_size_inches((10, 8))

In [71]:
final_features = ["AWEI", "NIR", "Slope", "Elevation"]

X_train, X_temp, y_train, y_temp = train_test_split(
    x[final_features], y, test_size=0.3, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=(1 / 3), stratify=y_temp
)

np.savez(
    "../data/asg3/complete_data.npz",
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    X_val=X_val,
    y_val=y_val,
)

In [72]:
data = np.load("../data/asg3/complete_data.npz")
(
    X_train,
    X_test,
    y_train,
    y_test,
    X_val,
    y_val,
) = (
    data["X_train"],
    data["X_test"],
    data["y_train"],
    data["y_test"],
    data["X_val"],
    data["y_val"],
)

train_data = lgb.Dataset(X_train, y_train, params={"feature_pre_filter": False})

X_val_early_stop, X_val_tune, y_val_early_stop, y_val_tune = train_test_split(
    X_val, y_val, test_size=0.5, shuffle=True, stratify=y_val
)

In [ ]:
forest.fit(X_train, y_train)

explainer = shap.GPUTreeExplainer(forest)
clustering = shap.utils.hclust(X_train, y_train)

explanation = explainer(X_test)
shap_values = explainer.shap_values(X_test)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 8))

axes[0].set_title("Feature Importance and Clustering \nBased on SHAP Values")
shap.plots.bar(
    explanation,
    clustering=clustering,
    max_display=len(x.columns),
    ax=axes[0],
    show=False,
)

shap.decision_plot(
    explainer.expected_value,
    shap_values[:10],
    feature_names=final_features,
    link="logit",
    show=False,
    auto_size_plot=False,
)
axes[1] = plt.gca()
axes[1].set_title("Complexity of Model for Making Prediction")
fig.set_dpi(1000)
fig.suptitle("Analysis for Selected Features", fontsize=16)
fig.tight_layout()
fig.set_size_inches((10, 8))

In [53]:
def evaluate_model(y_pred, y_test):
    precision, recall, f1, support = precision_recall_fscore_support(
        y_test, y_pred, average="binary"
    )
    accuracy = accuracy_score(y_test, lgb_y_pred)
    auc = roc_auc_score(y_test, lgb_y_pred)
    return [accuracy, precision, recall, f1, auc]

In [51]:
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

lr_y_pred = logistic_model.predict(X_test)
lr_results = evaluate_model(lr_y_pred, y_test):

NameError: name 'LogisticRegression' is not defined

In [75]:
lgb_model = lgb.cv(
    {
        "objective": "binary",
        "metric": "auc",
        "device_type": "cpu",
        "verbose": -1,
        "boosting_type": "gbdt",
        "deterministic": True,
    },
    train_data,
    nfold=5,
    stratified=True,
    shuffle=True,
    metrics="auc",
    return_cvbooster=True,
)
# Predict and evaluate
lgb_y_pred = (lgb_model["cvbooster"].predict(X_test)[0] > 0.5).astype("int")
lgb_results = evaluate_model(lgb_y_pred, y_test)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
tabnet = TabNetClassifier(device_name="cuda", verbose=0)
tabnet.fit(X_train, y_train, eval_set=[(X_val_early_stop, y_val_early_stop)])

tabnet_y_pred = tabnet.predict(X_test)
tabnet_results = evaluate_model(tabnet_y_pred, y_test)

In [ ]:
metrics_df = pd.DataFrame(
    {
        "Metric": ["Accuracy", "Precision", "Recall", "F1 Score", "AUC"],
        "Logistic Regression": lr_results,
        "LightGBM": lgb_results,
        "TabNet": tabnet_results,
    }
)

metrics_df.to_csv("../data/asg3/initial_results.csv", index=False)
metrics_df.style.background_gradient(axis=1)

In [2]:
def objective(trial):
    # Define the hyperparameters to tune
    params = {
        "objective": "binary",
        "metric": "auc",
        "device_type": "cpu",
        "verbose": -1,
        "boosting_type": "gbdt",
        "deterministic": True,
        "max_depth": trial.suggest_int("max_depth", 0, 15),
        "num_leaves": trial.suggest_int("num_leaves", 31, 1024),
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True),
        "num_iterations": trial.suggest_int("n_estimators", 100, 1000),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-6, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-6, 10.0, log=True),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 20, 800),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
    }

    # Create the LightGBM model
    model = lgb.cv(
        params,
        train_data,
        nfold=5,
        stratified=True,
        shuffle=True,
        metrics="auc",
        return_cvbooster=True,
    )

    y_pred = (model["cvbooster"].predict(X_test)[0] > 0.5).astype("int")
    auc = roc_auc_score(y_val_tune, y_pred)

    return auc


study = optuna.create_study(
    direction="maximize",
    study_name="LightGBM_study",
    storage="sqlite:///../data/asg3/lightgbm_optuna.db",
    load_if_exists=True,
    sampler=GPSampler(seed=0, deterministic_objective=True),
    pruner=optuna.pruners.SuccessiveHalvingPruner(),
)

n_trials = 3000


def ensure_n_trials(study, trial):
    if trial.number >= n_trials:
        study.stop()


study.optimize(objective, n_trials=n_trials, callbacks=[ensure_n_trials], n_jobs=-1)

/tmp/ipykernel_21685/3556903182.py:43: ExperimentalWarning: GPSampler is experimental (supported from v3.6.0). The interface can change in the future.
[I 2024-08-01 05:10:13,274] Using an existing study with name 'LightGBM_study' instead of creating a new one.


In [ ]:
from optuna.integration.lightgbm import LightGBMTunerCV

cv_study = optuna.create_study(
    study_name="LightGBM_tuner_cv",
    storage="sqlite:///../data/asg3/lightgbm_tuner_cv.db",
)

tuner = LightGBMTunerCV(
    {
        "objective": "binary",
        "metric": "auc",
        "device_type": "cpu",
        "verbose": -1,
        "boosting_type": "gbdt",
        "deterministic": True,
    },
    train_data,
    verbose_eval=False,
    study=cv_study,
    nfold=5,
    stratified=True,
    shuffle=True,
    show_progress_bar=True,
)
tuner.set_verbosity(-1)

tuner.run()

In [7]:
best_trial = study.trials_dataframe()["value"].idxmax()
trials = study.get_trials()

print(f"Best trial: {best_trial}")
print(f"Value: {trials[best_trial].value}")
print("Params: ")
for key, value in trials[best_trial].params.items():
    print(f"    {key}: {value}")

Best trial: 559
Value: 0.929425377499083
Params: 
    max_depth: 15
    num_leaves: 283
    learning_rate: 0.084350629755705
    n_estimators: 723
    lambda_l1: 1.1690787181558855
    lambda_l2: 0.9437281129035027
    feature_fraction: 0.8530414503922408
    min_child_samples: 181
    bagging_fraction: 0.7963514731926218
    bagging_freq: 6


In [17]:
best_params = trials[best_trial].params
tuned_lgb = lgb.train(
    {
        "objective": "binary",
        "metric": "auc",
        "device_type": "cpu",
        "verbose": -1,
        "boosting_type": "gbdt",
        "deterministic": True,
        **best_params,
    },
    train_data,
    valid_sets=[val_data_early_stop],
    callbacks=[lgb.early_stopping(stopping_rounds=20, verbose=False)],
)

In [19]:
# Evaluate the best model
tuned_lgb_y_pred = (tuned_lgb.predict(X_test) > 0.5).astype("int")
tuned_lgb_precision, tuned_lgb_recall, tuned_lgb_f1, tuned_lgb_support = (
    precision_recall_fscore_support(y_test, tuned_lgb_y_pred, average="binary")
)
tuned_lgb_accuracy = accuracy_score(y_test, tuned_lgb_y_pred)
tuned_lgb_auc = roc_auc_score(y_test, tuned_lgb_y_pred)

In [20]:
final_results = pd.read_csv("../data/asg3/initial_results.csv")
final_results.loc[-1] = [
    "Tuned_LightGBM",
    tuned_lgb_accuracy,
    tuned_lgb_precision,
    tuned_lgb_recall,
    tuned_lgb_f1,
    tuned_lgb_auc,
]

final_results.index += 1
final_results = final_results.sort_index()
final_results.style.background_gradient(axis=0)

,Model,Accuracy,Precision,Recall,F1 Score,AUC
0,Tuned_LightGBM,0.855401,0.876519,0.823401,0.849130,0.855033
1,Logistic Regression,0.791521,0.799062,0.772369,0.785489,0.791302
2,LightGBM,0.856977,0.878555,0.824576,0.850710,0.856605
3,TabNet,0.843620,0.913485,0.755078,0.826762,0.842604


In [ ]:
viz_model = dtreeviz.model(
    tuned_lgb,
    tree_index=1,
    X_train=X_train,
    y_train=y_train,
    feature_names=["AWEI", "NIR", "Slope", "Elevation"],
    target_name="Flooded",
    class_names=["Non-water", "Flooded"],
)
viz_model.view(x=X_train[1])

In [ ]:
Map

In [ ]:
bands = ["SR_B1", "SR_B2", "SR_B4", "SR_B5", "SR_B7", "z", "slope", "flooded"]

roi_geo_path = Path("../data/asg3/roi_geo.npy")

if Map.draw_last_feature:
    geometry = Map.draw_last_feature.geometry()
    bounds = geemap.ee_to_bbox(geometry)
    np.save(roi_geo_path, bounds)
    reduced_target_image = target_image.reduceResolution(
        reducer=ee.Reducer.mean(),
    )

    np.save(
        "../data/asg3/roi",
        geemap.ee_to_numpy(
            reduced_target_image,
            region=geometry,
            scale=250,
            bands=bands,
        ),
    )
    Map.remove_drawn_features()
else:
    if roi_geo_path.exists():
        bounds = np.load(roi_geo_path).tolist()
        roi_geo = ee.Geometry.Rectangle(coords=bounds)
        Map.centerObject(roi_geo, zoom=10)
        roi_geo = ee.FeatureCollection(roi_geo).style(
            fillColor="#3181cc33", color="red", width=1
        )
        Map.addLayer(roi_geo, {}, "ROI Geometry")

In [ ]:
data: np.ndarray = np.load("../data/asg3/roi.npy")
rows, cols, bands_len = data.shape

array_2d = data.reshape(-1, bands_len)
coordinates = [(row, col) for row in range(rows) for col in range(cols)]
data_with_coords = [
    {"x": x, "y": y, **{bands[i]: array_2d[idx, i] for i in range(bands_len)}}
    for idx, (x, y) in enumerate(coordinates)
]

features = {
    "SR_B1": "Blue",
    "SR_B2": "Green",
    "SR_B4": "NIR",
    "SR_B5": "SWIR_1",
    "SR_B7": "SWIR_2",
    "slope": "Slope",
    "z": "Elevation",
}

df_target_image = pd.DataFrame(data_with_coords).rename(columns=features)
x = df_target_image[features.values()]
y = df_target_image["flooded"]
x["AWEI"] = (
    x["Blue"] + 2.5 * x["Green"] - 1.5 * (x["NIR"] + x["SWIR_1"]) - 0.25 * x["SWIR_2"]
)

y_pred = tuned_lgb.predict(x)